# ZipVoice Vietnamese 2500h - Zero-Shot TTS on Google Colab

**Zero-shot voice cloning for Vietnamese** | 123M parameters | 24kHz output | T4 GPU optimized

📄 [Paper](https://arxiv.org/abs/2506.13053) • 🔗 [GitHub](https://github.com/k2-fsa/ZipVoice) • 🎵 [Demo](https://zipvoice.github.io)

## 1. Check GPU (T4 Required)

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Setup: Clone Repositories & Download Reference Audio

In [ ]:
# Setup: Clone repositories and download reference audio
import os
import subprocess

print("="*80)
print("SETUP: Cloning repositories and downloading reference audio")
print("="*80)

# 1. Clone ZipVoice repository from GitHub
print("\n[1/3] Cloning ZipVoice repository from GitHub...")
ZIPVOICE_REPO = "/content/ZipVoice"

if os.path.exists(ZIPVOICE_REPO):
    print(f"✓ ZipVoice repository already exists at {ZIPVOICE_REPO}")
else:
    try:
        subprocess.run([
            "git", "clone", 
            "https://github.com/k2-fsa/ZipVoice.git",
            ZIPVOICE_REPO
        ], check=True, capture_output=True)
        print(f"✓ Successfully cloned ZipVoice repository to {ZIPVOICE_REPO}")
    except subprocess.CalledProcessError as e:
        print(f"✗ Failed to clone ZipVoice repository: {e}")
        raise

# 2. Clone Vietnamese model from Hugging Face
print("\n[2/3] Cloning Vietnamese model from Hugging Face...")
MODEL_DIR = "/content/ZipVoice-Vietnamese-2500h"

if os.path.exists(MODEL_DIR):
    print(f"✓ Model directory already exists at {MODEL_DIR}")
else:
    try:
        subprocess.run([
            "git", "clone",
            "https://huggingface.co/hynt/ZipVoice-Vietnamese-2500h",
            MODEL_DIR
        ], check=True, capture_output=True)
        print(f"✓ Successfully cloned model to {MODEL_DIR}")
    except subprocess.CalledProcessError as e:
        print(f"✗ Failed to clone model repository: {e}")
        raise

# 3. Download reference audio from Google Drive
print("\n[3/3] Downloading reference audio from Google Drive...")
REFERENCE_AUDIO = "/content/reference_audio_3s.wav"
REFERENCE_TEXT = "Chào mừng quý vị và các bạn đã quay trở lại với kênh của chúng tôi."

# Google Drive direct download link for reference_audio_3s.wav
# File ID from: https://drive.google.com/file/d/1PISVrfUSWRSj-zMrgmV-ABC5LTcLju-Z/view?usp=sharing
AUDIO_FILE_ID = "1PISVrfUSWRSj-zMrgmV-ABC5LTcLju-Z"

if os.path.exists(REFERENCE_AUDIO):
    print(f"✓ Reference audio already exists at {REFERENCE_AUDIO}")
else:
    try:
        # Install gdown if not already installed
        try:
            import gdown
        except ImportError:
            print("Installing gdown...")
            subprocess.run(["pip", "install", "-q", "gdown"], check=True)
            import gdown
        
        # Download the reference audio file
        url = f"https://drive.google.com/uc?id={AUDIO_FILE_ID}"
        gdown.download(url, REFERENCE_AUDIO, quiet=False)
        print(f"✓ Successfully downloaded reference audio to {REFERENCE_AUDIO}")
    except Exception as e:
        print(f"⚠️  Could not download reference audio: {e}")
        print("You can manually upload reference_audio_3s.wav using the cell below.")

# Set global variables
PROMPT_WAV = REFERENCE_AUDIO
PROMPT_TEXT = REFERENCE_TEXT

print("\n" + "="*80)
print("SETUP COMPLETE")
print("="*80)
print(f"✓ ZipVoice repository: {ZIPVOICE_REPO}")
print(f"✓ Model directory: {MODEL_DIR}")
print(f"✓ Reference audio: {PROMPT_WAV}")
print(f"✓ Reference text: {PROMPT_TEXT}")

# Verify critical files
print("\n" + "="*80)
print("VERIFICATION")
print("="*80)

print("\nChecking model files...")
required_files = ["config.json", "tokens.txt", "iter-525000-avg-2.pt"]
all_found = True
for file in required_files:
    file_path = f"{MODEL_DIR}/{file}"
    if os.path.exists(file_path):
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"✓ Found {file} ({size_mb:.1f} MB)")
    else:
        print(f"✗ Missing {file}")
        all_found = False

if all_found:
    print("\n🎉 All files ready! You can proceed to the next step.")
else:
    print("\n⚠️  Some files are missing. Please check the repositories.")

## 3. Install Dependencies

In [ ]:
# Change to ZipVoice directory and install dependencies
%cd {ZIPVOICE_REPO}

# Install required packages
print("Installing ZipVoice dependencies...")
!pip install -q -r requirements.txt
print("✓ Dependencies installed!")

# Install k2 for efficient inference
print("\nInstalling k2 for optimized inference...")
!pip install -q k2 -f https://k2-fsa.github.io/k2/cuda.html

# Verify k2 installation
try:
    import k2
    print(f"✓ k2 installed successfully: {k2.__file__}")
except ImportError:
    print("⚠ k2 installation failed, but you can still run inference without it (slower)")

## 4. Load Model & Initialize

In [ ]:
# Load model and initialize all components
import torch
import json
from zipvoice.models.zipvoice import ZipVoice
from zipvoice.utils.checkpoint import load_checkpoint
from zipvoice.tokenizer.tokenizer import EspeakTokenizer
from zipvoice.utils.feature import VocosFbank
from vocos import Vocos

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load configuration
config_path = f"{MODEL_DIR}/config.json"
print(f"\nLoading config from: {config_path}")
with open(config_path, "r") as f:
    config = json.load(f)

# Count tokens
tokens_path = f"{MODEL_DIR}/tokens.txt"
with open(tokens_path, "r") as f:
    num_tokens = len(f.readlines())

print(f"Model config loaded:")
print(f"  - Feature dim: {config['model']['feat_dim']}")
print(f"  - Sampling rate: {config['feature']['sampling_rate']} Hz")
print(f"  - Number of tokens: {num_tokens}")

# Initialize model
print("\nInitializing ZipVoice model...")
model = ZipVoice(**config["model"], vocab_size=num_tokens)

# Load checkpoint
checkpoint_path = f"{MODEL_DIR}/iter-525000-avg-2.pt"
print(f"Loading checkpoint from: {checkpoint_path}")
load_checkpoint(checkpoint_path, model)
model.to(device).eval()
print("✓ Model loaded!")

# Initialize tokenizer
print("\nInitializing tokenizer for Vietnamese...")
tokenizer = EspeakTokenizer(token_file=f"{MODEL_DIR}/tokens.txt", lang="vi")
print("✓ Tokenizer initialized!")

# Initialize feature extractor
print("\nInitializing feature extractor...")
feature_extractor = VocosFbank()
print("✓ Feature extractor initialized!")

# Initialize vocoder
print("\nLoading Vocos vocoder...")
vocoder = Vocos.from_pretrained("charactr/vocos-mel-24khz")
vocoder.to(device).eval()
print("✓ Vocoder loaded!")

print("\n" + "="*60)
print("All components loaded successfully!")
print("="*60)

## 6.1. Import Preprocessing Utilities

In [ ]:
# Import preprocessing utilities
import datetime as dt
import logging
from zipvoice.utils.infer import (
    add_punctuation, batchify_tokens, chunk_tokens_punctuation,
    cross_fade_concat, load_prompt_wav, remove_silence, rms_norm,
)

print("✓ Preprocessing utilities imported")

## 6.2. Define Original Inference Function

In [ ]:
def generate_sentence(
    save_path: str,
    prompt_text: str,
    prompt_wav: str,
    text: str,
    model,
    vocoder,
    tokenizer,
    feature_extractor,
    device,
    num_step: int = 16,
    guidance_scale: float = 1.0,
    speed: float = 1.0,
    t_shift: float = 0.5,
    target_rms: float = 0.1,
    feat_scale: float = 0.1,
    sampling_rate: int = 24000,
    max_duration: float = 100,
    remove_long_sil: bool = False,
):
    """
    Generate waveform of a text based on a given prompt waveform and its transcription.
    This function includes full preprocessing pipeline from original ZipVoice repository:
    1. Remove silences from prompt audio
    2. Add punctuation if missing
    3. Chunk text at punctuation marks
    4. Process chunks in batches
    5. Cross-fade audio chunks
    6. Remove final silences
    
    Adapted from: ZipVoice/zipvoice/bin/infer_zipvoice.py
    """
    
    # Suppress torchaudio deprecation warnings
    import warnings
    warnings.filterwarnings('ignore', category=UserWarning, module='torchaudio')
    
    # Step 1: Load and clean prompt audio
    prompt_wav_tensor = load_prompt_wav(prompt_wav, sampling_rate=sampling_rate)
    
    # Remove edge and long silences in the prompt wav.
    # Add 0.2s trailing silence to avoid leaking prompt to generated speech.
    prompt_wav_tensor = remove_silence(
        prompt_wav_tensor, sampling_rate, only_edge=False, trail_sil=200
    )
    
    prompt_wav_tensor, prompt_rms = rms_norm(prompt_wav_tensor, target_rms)
    prompt_duration = prompt_wav_tensor.shape[-1] / sampling_rate
    
    if prompt_duration > 20:
        logging.warning(
            f"Given prompt wav is too long ({prompt_duration}s). "
            f"Please provide a shorter one (1-3 seconds is recommended)."
        )
    elif prompt_duration > 10:
        logging.warning(
            f"Given prompt wav is long ({prompt_duration}s). "
            f"It will lead to slower inference speed and possibly worse speech quality."
        )
    
    # Step 2: Extract features from cleaned prompt
    prompt_features = feature_extractor.extract(
        prompt_wav_tensor, sampling_rate=sampling_rate
    ).to(device)
    prompt_features = prompt_features.unsqueeze(0) * feat_scale
    
    # Step 3: Add punctuation if missing
    text = add_punctuation(text)
    prompt_text = add_punctuation(prompt_text)
    
    # Step 4: Tokenize text (string tokens), punctuations preserved
    tokens_str = tokenizer.texts_to_tokens([text])[0]
    prompt_tokens_str = tokenizer.texts_to_tokens([prompt_text])[0]
    
    # Step 5: Chunk text at punctuation marks
    # Calculate token duration from prompt
    token_duration = (prompt_wav_tensor.shape[-1] / sampling_rate) / (
        len(prompt_tokens_str) * speed
    )
    # Each chunk should be ~25 seconds
    max_tokens = int((25 - prompt_duration) / token_duration)
    chunked_tokens_str = chunk_tokens_punctuation(tokens_str, max_tokens=max_tokens)
    
    # Step 6: Convert string tokens to IDs
    chunked_tokens = tokenizer.tokens_to_token_ids(chunked_tokens_str)
    prompt_tokens = tokenizer.tokens_to_token_ids([prompt_tokens_str])
    
    # Step 7: Batchify chunks for efficient processing
    tokens_batches, chunked_index = batchify_tokens(
        chunked_tokens, max_duration, prompt_duration, token_duration
    )
    
    # Step 8: Generate features for each batch
    chunked_features = []
    start_t = dt.datetime.now()
    
    for batch_tokens in tokens_batches:
        batch_prompt_tokens = prompt_tokens * len(batch_tokens)
        batch_prompt_features = prompt_features.repeat(len(batch_tokens), 1, 1)
        batch_prompt_features_lens = torch.full(
            (len(batch_tokens),), prompt_features.size(1), device=device
        )
        
        # Generate features
        (
            pred_features,
            pred_features_lens,
            pred_prompt_features,
            pred_prompt_features_lens,
        ) = model.sample(
            tokens=batch_tokens,
            prompt_tokens=batch_prompt_tokens,
            prompt_features=batch_prompt_features,
            prompt_features_lens=batch_prompt_features_lens,
            speed=speed,
            t_shift=t_shift,
            duration="predict",
            num_step=num_step,
            guidance_scale=guidance_scale,
        )
        
        # Postprocess predicted features
        pred_features = pred_features.permute(0, 2, 1) / feat_scale  # (B, C, T)
        chunked_features.append((pred_features, pred_features_lens))
    
    # Step 9: Convert features to waveforms with vocoder
    chunked_wavs = []
    start_vocoder_t = dt.datetime.now()
    
    for pred_features, pred_features_lens in chunked_features:
        batch_wav = []
        for i in range(pred_features.size(0)):
            wav = (
                vocoder.decode(pred_features[i][None, :, : pred_features_lens[i]])
                .squeeze(1)
                .clamp(-1, 1)
            )
            # Adjust wav volume if necessary
            if prompt_rms < target_rms:
                wav = wav * prompt_rms / target_rms
            batch_wav.append(wav)
        chunked_wavs.extend(batch_wav)
    
    # Finish model generation timing
    t = (dt.datetime.now() - start_t).total_seconds()
    
    # Step 10: Merge chunked wavs with cross-fading
    indexed_chunked_wavs = [
        (index, wav) for index, wav in zip(chunked_index, chunked_wavs)
    ]
    sequential_indexed_chunked_wavs = sorted(indexed_chunked_wavs, key=lambda x: x[0])
    sequential_chunked_wavs = [
        sequential_indexed_chunked_wavs[i][1]
        for i in range(len(sequential_indexed_chunked_wavs))
    ]
    final_wav = cross_fade_concat(
        sequential_chunked_wavs, fade_duration=0.1, sample_rate=sampling_rate
    )
    
    # Step 11: Remove final silences
    final_wav = remove_silence(
        final_wav, sampling_rate, only_edge=(not remove_long_sil), trail_sil=0
    )
    
    # Calculate processing time metrics
    t_no_vocoder = (start_vocoder_t - start_t).total_seconds()
    t_vocoder = (dt.datetime.now() - start_vocoder_t).total_seconds()
    wav_seconds = final_wav.shape[-1] / sampling_rate
    rtf = t / wav_seconds
    rtf_no_vocoder = t_no_vocoder / wav_seconds
    rtf_vocoder = t_vocoder / wav_seconds
    
    metrics = {
        "t": t,
        "t_no_vocoder": t_no_vocoder,
        "t_vocoder": t_vocoder,
        "wav_seconds": wav_seconds,
        "rtf": rtf,
        "rtf_no_vocoder": rtf_no_vocoder,
        "rtf_vocoder": rtf_vocoder,
        "num_chunks": len(chunked_tokens),
        "prompt_duration": prompt_duration,
    }
    
    # Save the generated audio
    import torchaudio
    torchaudio.save(save_path, final_wav.cpu(), sample_rate=sampling_rate)
    
    return metrics

print("✓ generate_sentence() function defined successfully!")
print("\nThis function includes full preprocessing pipeline:")
print("  1. Prompt audio cleaning (silence removal)")
print("  2. Auto-punctuation addition")
print("  3. Text chunking at punctuation marks")
print("  4. Batch processing for efficiency")
print("  5. Cross-fading between audio chunks")
print("  6. Final silence removal")

## 6.3. Generate with Original Pipeline

In [ ]:
# Generate Vietnamese speech
import torch

# === TEXT INPUT FOR AUTOMATION ===
TEXT_TO_SYNTHESIZE = "Xin chào, tôi là trợ lý ảo của bạn. Tôi có thể giúp gì cho bạn hôm nay?"
# === END TEXT INPUT ===

OUTPUT_WAV = "/content/output_vietnamese.wav"

# Parameters
NUM_STEP = 8
SPEED = 1.0
REMOVE_LONG_SIL = False
MAX_DURATION = 100

print("="*70)
print("GENERATING SPEECH")
print("="*70)
print(f"Text: {TEXT_TO_SYNTHESIZE[:60]}...")
print(f"Output: {OUTPUT_WAV}")
print(f"Params: num_step={NUM_STEP}, speed={SPEED}")
print("="*70)

print("✓ GENERATION_STARTED")

# Generate
with torch.inference_mode():
    metrics = generate_sentence(
        save_path=OUTPUT_WAV,
        prompt_text=PROMPT_TEXT,
        prompt_wav=PROMPT_WAV,
        text=TEXT_TO_SYNTHESIZE,
        model=model,
        vocoder=vocoder,
        tokenizer=tokenizer,
        feature_extractor=feature_extractor,
        device=device,
        num_step=NUM_STEP,
        guidance_scale=1.0,
        speed=SPEED,
        t_shift=0.5,
        target_rms=0.1,
        feat_scale=0.1,
        sampling_rate=config["feature"]["sampling_rate"],
        max_duration=MAX_DURATION,
        remove_long_sil=REMOVE_LONG_SIL,
    )

print("="*70)
print("✅ COMPLETE")
print("="*70)
print(f"Duration: {metrics['wav_seconds']:.2f}s | Time: {metrics['t']:.2f}s | RTF: {metrics['rtf']:.3f}x")
print(f"Chunks: {metrics['num_chunks']} | Saved: {OUTPUT_WAV}")
print("="*70)

print("✓ GENERATION_COMPLETE")

## 7. Play Audio

In [ ]:
# Play the generated audio
from IPython.display import Audio, display

print("Reference:")
display(Audio(PROMPT_WAV))

print("\nGenerated:")
display(Audio(OUTPUT_WAV))

## 8. Download Generated Audio

In [ ]:
# Download the generated audio file
from google.colab import files

print("Downloading generated audio...")
files.download(OUTPUT_WAV)
print(f"✓ Download complete: {OUTPUT_WAV}")